In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, desc, row_number, to_date, year
from pyspark.sql.window import Window
import time

# Δημιουργία Spark Session
spark = SparkSession.builder \
    .appName("Query 2 - Dataframe API") \
    .getOrCreate()

# Μέτρηση χρόνου
start_time_df = time.time()

# Φόρτωση δεδομένων
crime_2010_2019_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_2020_present_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"

crime_df_2010_2019 = spark.read.csv(crime_2010_2019_path, header=True, inferSchema=True)
crime_df_2020_present = spark.read.csv(crime_2020_present_path, header=True, inferSchema=True)
crime_df = crime_df_2010_2019.union(crime_df_2020_present)

# Φιλτράρισμα κλεισμένων υποθέσεων
crime_data = crime_df.withColumn(
    "is_closed",
    when(col("Status Desc").isin("UNK", "Invest Cont"), 0).otherwise(1)
)

# Μετατροπή της στήλης DATE OCC σε ημερομηνία και εξαγωγή του έτους
crime_data = crime_data.withColumn("year", year(to_date(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a")))

# Υπολογισμός ποσοστού κλεισμένων υποθέσεων ανά έτος και τμήμα
closed_case_rate = crime_data.groupBy(
    "year", 
    "AREA NAME"
).agg(
    (count(when(col("is_closed") == 1, 1)) / count("*") * 100).alias("closed_case_rate")
).select(
    "year", 
    col("AREA NAME").alias("precinct"),
    "closed_case_rate"
)

# Καθορισμός παραθύρου για την κατάταξη
window_spec = Window.partitionBy("year").orderBy(desc("closed_case_rate"))

# Προσθήκη στήλης κατάταξης
ranked_data = closed_case_rate.withColumn("#", row_number().over(window_spec))

# Επιλογή των 3 κορυφαίων τμημάτων ανά έτος
top_3_per_year = ranked_data.filter(col("#") <= 3).orderBy("year", "#")

# Εμφάνιση αποτελεσμάτων
top_3_per_year.show(45)

end_time_df = time.time()

df_execution_time = end_time_df - start_time_df

print(f"DataFrame API Execution Time: {df_execution_time:.2f} seconds")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
319,application_1738075734771_0320,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+---+
|year|   precinct|  closed_case_rate|  #|
+----+-----------+------------------+---+
|2010|    Rampart| 32.84713448949121|  1|
|2010|    Olympic|31.515289821999087|  2|
|2010|     Harbor| 29.36028339237341|  3|
|2011|    Olympic|35.040060090135206|  1|
|2011|    Rampart|  32.4964471814306|  2|
|2011|     Harbor| 28.51336246316431|  3|
|2012|    Olympic| 34.29708533302119|  1|
|2012|    Rampart| 32.46000463714352|  2|
|2012|     Harbor|29.509585848956675|  3|
|2013|    Olympic| 33.58217940999398|  1|
|2013|    Rampart|  32.1060382916053|  2|
|2013|     Harbor|29.723638951488557|  3|
|2014|   Van Nuys|  32.0215235281705|  1|
|2014|West Valley| 31.49754809505847|  2|
|2014|    Mission|31.224939855653567|  3|
|2015|   Van Nuys|32.265140677157845|  1|
|2015|    Mission|30.463762673676303|  2|
|2015|   Foothill|30.353001803658852|  3|
|2016|   Van Nuys|32.194518462124094|  1|
|2016|West Valley| 31.40146437042384|  2|
|2016|   Foothill|29.9086472281316

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, desc, row_number, to_date, year
from pyspark.sql.window import Window
import time

# Δημιουργία Spark Session
spark = SparkSession.builder \
    .appName("Query 2 - SQL API") \
    .getOrCreate()

# Μέτρηση χρόνου
start_time_sql = time.time()

# Φόρτωση δεδομένων
crime_2010_2019_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_2020_present_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"

crime_df_2010_2019 = spark.read.csv(crime_2010_2019_path, header=True, inferSchema=True)
crime_df_2020_present = spark.read.csv(crime_2020_present_path, header=True, inferSchema=True)
crime_df = crime_df_2010_2019.union(crime_df_2020_present)

# Φιλτράρισμα ανοιχτών και κλεισμένων υποθέσεων
crime_data = crime_df.withColumn(
    "is_closed",
    when(col("Status Desc").isin("UNK", "Invest Cont"), 0).otherwise(1)
)

# Μετατροπή της στήλης DATE OCC σε ημερομηνία και εξαγωγή του έτους
crime_data = crime_data.withColumn("year", year(to_date(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a")))

# Δημιουργία προσωρινού view
crime_data.createOrReplaceTempView("crime_data")

# Υλοποίηση με SQL
query = """
WITH ranked_data AS (
    SELECT 
        year, 
        `AREA NAME` AS precinct, 
        (SUM(CASE WHEN `Status Desc` NOT IN ('UNK', 'Invest Cont') THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS closed_case_rate,
        RANK() OVER (PARTITION BY year ORDER BY (SUM(CASE WHEN `Status Desc` NOT IN ('UNK', 'Invest Cont') THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) DESC) AS `#`
    FROM crime_data
    GROUP BY year, `AREA NAME`
)
SELECT year, precinct, closed_case_rate, `#`
FROM ranked_data
WHERE `#` <= 3
ORDER BY year, `#`
"""

top_3_per_year_sql = spark.sql(query)

# Εμφάνιση αποτελεσμάτων
top_3_per_year_sql.show(45)

end_time_sql = time.time()

sql_execution_time = end_time_sql - start_time_sql

print(f"SQL API Execution Time: {sql_execution_time:.2f} seconds")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
324,application_1738075734771_0325,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+-----------------+---+
|year|   precinct| closed_case_rate|  #|
+----+-----------+-----------------+---+
|2010|    Rampart|32.84713448949121|  1|
|2010|    Olympic|31.51528982199909|  2|
|2010|     Harbor|29.36028339237341|  3|
|2011|    Olympic|35.04006009013520|  1|
|2011|    Rampart|32.49644718143060|  2|
|2011|     Harbor|28.51336246316431|  3|
|2012|    Olympic|34.29708533302119|  1|
|2012|    Rampart|32.46000463714352|  2|
|2012|     Harbor|29.50958584895668|  3|
|2013|    Olympic|33.58217940999398|  1|
|2013|    Rampart|32.10603829160530|  2|
|2013|     Harbor|29.72363895148855|  3|
|2014|   Van Nuys|32.02152352817050|  1|
|2014|West Valley|31.49754809505847|  2|
|2014|    Mission|31.22493985565357|  3|
|2015|   Van Nuys|32.26514067715784|  1|
|2015|    Mission|30.46376267367630|  2|
|2015|   Foothill|30.35300180365885|  3|
|2016|   Van Nuys|32.19451846212410|  1|
|2016|West Valley|31.40146437042384|  2|
|2016|   Foothill|29.90864722813165|  3|
|2017|   Van Nuy

In [2]:
s3_path = "s3://groups-bucket-dblab-905418150721/group38/"
crime_df.coalesce(1).write.mode("overwrite").parquet(s3_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, desc, row_number, to_date, year
from pyspark.sql.window import Window
import time

# Δημιουργία Spark Session
spark = SparkSession.builder \
    .appName("Query 2 - Dataframe API (.parquet)") \
    .getOrCreate()

# Μέτρηση χρόνου
start_time_df = time.time()

# Φόρτωση δεδομένων
s3_path = "s3://groups-bucket-dblab-905418150721/group38/"
crime_df = spark.read.parquet(s3_path)

# Φιλτράρισμα ανοιχτών και κλεισμένων υποθέσεων
crime_data = crime_df.withColumn(
    "is_closed",
    when(col("Status Desc").isin("UNK", "Invest Cont"), 0).otherwise(1)
)

# Μετατροπή της στήλης DATE OCC σε ημερομηνία και εξαγωγή του έτους
crime_data = crime_data.withColumn("year", year(to_date(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a")))

# Υπολογισμός ποσοστού κλεισμένων υποθέσεων ανά έτος και τμήμα
closed_case_rate = crime_data.groupBy(
    "year", 
    "AREA NAME"
).agg(
    (count(when(col("is_closed") == 1, 1)) / count("*") * 100).alias("closed_case_rate")
).select(
    "year", 
    col("AREA NAME").alias("precinct"),
    "closed_case_rate"
)

# Καθορισμός παραθύρου για την κατάταξη
window_spec = Window.partitionBy("year").orderBy(desc("closed_case_rate"))

# Προσθήκη στήλης κατάταξης
ranked_data = closed_case_rate.withColumn("#", row_number().over(window_spec))

# Επιλογή των 3 κορυφαίων τμημάτων ανά έτος
top_3_per_year = ranked_data.filter(col("#") <= 3).orderBy("year", "#")

# Εμφάνιση αποτελεσμάτων
top_3_per_year.show(45)

end_time_df = time.time()

df_execution_time = end_time_df - start_time_df

print(f"DataFrame API Execution Time (.parquet): {df_execution_time:.2f} seconds")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
330,application_1738075734771_0331,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+---+
|year|   precinct|  closed_case_rate|  #|
+----+-----------+------------------+---+
|2010|    Rampart| 32.84713448949121|  1|
|2010|    Olympic|31.515289821999087|  2|
|2010|     Harbor| 29.36028339237341|  3|
|2011|    Olympic|35.040060090135206|  1|
|2011|    Rampart|  32.4964471814306|  2|
|2011|     Harbor| 28.51336246316431|  3|
|2012|    Olympic| 34.29708533302119|  1|
|2012|    Rampart| 32.46000463714352|  2|
|2012|     Harbor|29.509585848956675|  3|
|2013|    Olympic| 33.58217940999398|  1|
|2013|    Rampart|  32.1060382916053|  2|
|2013|     Harbor|29.723638951488557|  3|
|2014|   Van Nuys|  32.0215235281705|  1|
|2014|West Valley| 31.49754809505847|  2|
|2014|    Mission|31.224939855653567|  3|
|2015|   Van Nuys|32.265140677157845|  1|
|2015|    Mission|30.463762673676303|  2|
|2015|   Foothill|30.353001803658852|  3|
|2016|   Van Nuys|32.194518462124094|  1|
|2016|West Valley| 31.40146437042384|  2|
|2016|   Foothill|29.9086472281316